In [1]:
import sys
sys.path.append('../')

import tquant as tq
from engine.globalconfig import GlobalConfig
from engine.loader.curveload import MarketDataLoader

from datetime import date, datetime
import tensorflow as tf
import pandas as pd

# Settings

In [2]:
config = GlobalConfig.from_json()
mdl = MarketDataLoader(config)

In [3]:
tq.Settings.evaluation_date

datetime.date(2023, 9, 29)

In [4]:
calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

# Market

In [5]:
rates = mdl.ir_eur_curve_estr['quote'].values
times = mdl.ir_eur_curve_estr['daycount'].values/365

rates_6m = mdl.ir_eur_curve_6m['quote'].values
times_6m = mdl.ir_eur_curve_6m['daycount'].values/365

In [6]:
estr_curve = tq.RateCurve(times, rates)
eur6m_curve = tq.RateCurve(times_6m, rates_6m)

# FRA

In [9]:
evaluation_date = tq.Settings.evaluation_date
calendar = tq.TARGET()
index = tq.IborIndex("EUR6M", calendar, 6, tq.TimeUnit.Months)
d1 = calendar.advance(evaluation_date, 3, tq.TimeUnit.Months, tq.BusinessDayConvention.ModifiedFollowing)
d2 = calendar.advance(d1, 6, tq.TimeUnit.Months, tq.BusinessDayConvention.ModifiedFollowing)
fra = tq.ForwardRateAgreement(d1,d2,tq.Position.Long, 0.03, 100, index, daycounter, calendar)

In [10]:
fra_engine = tq.FraDiscountingEngine(fra)

In [11]:
fra_engine.price(estr_curve, eur6m_curve, evaluation_date)

<tf.Tensor: shape=(), dtype=float64, numpy=0.5626463856388138>

# FRA QuantLib

In [12]:
import QuantLib as ql

In [13]:
today = ql.Date.from_date(evaluation_date)
ql.Settings.instance().evaluationDate = today

calendar_ql = ql.TARGET()

# curva 6m
date_6m_ql = []
for date_py in mdl.ir_eur_discount_6m['maturity_date']:
    date_6m_ql.append(ql.Date.from_date(date_py))
dfs_6m = mdl.ir_eur_discount_6m['market_quote'].to_list()
yieldTermStructure_6m = ql.DiscountCurve(date_6m_ql, dfs_6m, ql.Actual360(), ql.TARGET())
yieldTermStructure_6m.enableExtrapolation()
yieldTermStructure_6m_Handle = ql.YieldTermStructureHandle(yieldTermStructure_6m)

# curva estr
date_estr_ql = []
for date_py in mdl.ir_eur_discount_estr['maturity_date']:
    date_estr_ql.append(ql.Date.from_date(date_py))
dfs_estr = mdl.ir_eur_discount_estr['market_quote'].to_list()
yieldTermStructure_estr = ql.DiscountCurve(date_estr_ql, dfs_estr, ql.Actual360(), ql.TARGET())
yieldTermStructure_estr.enableExtrapolation()
yieldTermStructure_estr_Handle = ql.YieldTermStructureHandle(yieldTermStructure_estr)

In [14]:
startDate = calendar_ql.advance(today, ql.Period('3M'), ql.ModifiedFollowing)
maturityDate = calendar_ql.advance(startDate, ql.Period('6M'), ql.ModifiedFollowing)
index_ql = ql.Euribor6M(yieldTermStructure_6m_Handle)

In [15]:
date_6m_ql = []
for date_py in mdl.ir_eur_discount_6m['maturity_date']:
    date_6m_ql.append(ql.Date.from_date(date_py))
dfs_6m = mdl.ir_eur_discount_6m['market_quote'].to_list()
yieldTermStructure_6m = ql.DiscountCurve(date_6m_ql, dfs_6m, ql.Actual360(), ql.TARGET())
yieldTermStructure_6m.enableExtrapolation()

In [16]:
fra = ql.ForwardRateAgreement(startDate, maturityDate, ql.Position.Long, 0.03, 100, index_ql, yieldTermStructure_estr_Handle)
print('NPV:', fra.NPV())

NPV: 0.5270891161865767


# check 

In [23]:
# discount factor un po' differenti rispetto a ql, dovuto a curva non ancora implementata

In [20]:
yieldTermStructure_estr.discount(0.75)

0.971254410780627

In [19]:
estr_curve.discount(0.75)

<tf.Tensor: shape=(), dtype=float64, numpy=0.9708629540061835>

In [39]:
# forward rates stessa cosa

In [38]:
yieldTermStructure_6m.forwardRate(startDate, maturityDate,ql.Actual360(), ql.Simple).rate()

0.040746360270149884

In [25]:
eur6m_curve.forward_rate(d1, d2, daycounter, evaluation_date)

<tf.Tensor: shape=(), dtype=float64, numpy=0.04146692262760461>